In [1]:
import pysplit
import pandas as pd
import numpy as np
from wrf import getvar, get_basemap, latlon_coords
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import seaborn as sns
import argparse
import matplotlib as mpl
import xarray as xr
from itertools import product
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import shapely.geometry as sgeom
from shapely.ops import unary_union
from shapely.prepared import prep
from mpl_toolkits.basemap import Basemap
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd
import shapely
from descartes import PolygonPatch
from shapely.geometry import Point

In [2]:
can = gpd.GeoDataFrame.from_file("/glade/work/molina/DATA/ocean_shapefiles/ne_50m_geography_marine_polys.shp")

#goa
goa1 = can[can['name']== 'Gulf of Alaska']
goa2 = can[can['name']== 'North Pacific Ocean']
goa3 = can[can['name']== 'Cook Inlet']
#bs
bs1 = can[can['name']== 'Bering Sea']
bs2 = can[can['name']== 'Bristol Bay']
#ao
ao1 = can[can['name']== 'Arctic Ocean']
ao2 = can[can['name']== 'Beaufort Sea']
ao3 = can[can['name']== 'Chukchi Sea']
ao4 = can[can['name']== 'Amundsen Gulf']

In [3]:
land_shp_fname = shpreader.natural_earth(resolution='50m', category='physical', name='land')
land_geom = unary_union(list(shpreader.Reader(land_shp_fname).geometries()))
land = prep(land_geom)
def is_land(x, y):
    return land.contains(sgeom.Point(x, y))

In [4]:
which_climate='hist'
ds1=xr.open_dataset(f"/glade/scratch/molina/basile/traj_files/trajfile_jun_subregion1_{which_climate}.nc").frequency

In [5]:
wrffile = Dataset('/glade/scratch/molina/basile/wrfout_d01_2002-12-30_18:00:00.nc4')
tfile = getvar(wrffile, 'tk')
m = get_basemap(tfile)
lats, lons = latlon_coords(tfile)

In [6]:
x, y = np.meshgrid(ds1.lons.values, ds1.lats.values)
xnew, ynew = m(x, y, inverse=True)

In [7]:
themask=np.zeros(ds1.shape)
goamask=np.zeros(ds1.shape)
bsmask=np.zeros(ds1.shape)
aomask=np.zeros(ds1.shape)

nonemask=np.zeros(ds1.shape)

In [8]:
AO_others=[[44, 1],
 [44, 2],
 [45, 0],
 [45, 1],
 [45, 2],
 [45, 3],
 [45, 4],
 [45, 5],
 [46, 0],
 [46, 1],
 [46, 2],
 [46, 3],
 [46, 4],
 [46, 5],
 [47, 0],
 [47, 1],
 [47, 2],
 [47, 3],
 [47, 4],
 [47, 5],
 [47, 45],
 [46, 45],
 [46, 47],
 [46, 51],
 [45, 49],
 [44, 48],
 [43, 51],
 [42, 50],
 [42, 49],
 [37, 37],
 [37, 36],
 [36, 36],
 [36, 35],
 [36, 34],
 [30, 14],
 [30, 15],
 [30, 16],
 [31, 14]]

In [9]:
BS_others=[[33, 0],
 [32, 0],
 [32, 1],
 [32, 2],
 [31, 0],
 [31, 1],
 [31, 2],
 [30, 0],
 [30, 1],
 [30, 2],
 [30, 3],
 [29, 0],
 [29, 1],
 [29, 2],
 [29, 3],
 [28, 0],
 [28, 1],
 [26, 12],
 [26, 13],
 [26, 14],
 [26, 15],
 [25, 12],
 [25, 13],
 [25, 14],
 [25, 15],
 [24, 12],
 [24, 13],
 [24, 14],
 [24, 15],
 [18, 10]]

In [10]:
GOA_others=[[3, 51],
 [4, 50],
 [4, 51],
 [5, 49],
 [5, 50],
 [5, 51],
 [6, 48],
 [6, 49],
 [6, 50],
 [6, 51],
 [7, 49],
 [7, 50],
 [7, 51],
 [8, 48],
 [8, 51],
 [9, 45],
 [9, 46],
 [9, 47],
 [9, 48],
 [10, 47],
 [11, 44],
 [11, 45],
 [11, 46],
 [12, 45],
 [13, 42],
 [13, 43],
 [14, 41],
 [15, 28],
 [16, 28],
 [16, 29],
 [16, 30],
 [17, 29]]

In [11]:
none_points=[]

for i, j in product(range(0,themask.shape[0]), range(0,themask.shape[1])):
    
    if is_land(xnew[i,j], ynew[i,j]):
        themask[i,j]=1.

    elif goa1.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        goamask[i,j]=1.
        #print('Gulf of Alaska')
    elif goa2.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        goamask[i,j]=1.
        #print('Gulf of Alaska')
    elif goa3.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        goamask[i,j]=1.
        #print('Gulf of Alaska')
        
    elif bs1.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        bsmask[i,j]=1.
        #print('Bering Sea')
    elif bs2.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        bsmask[i,j]=1.
        #print('Bering Sea')
        
    elif ao1.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        aomask[i,j]=1.
        #print('Arctic Ocean')
    elif ao2.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        aomask[i,j]=1.
        #print('Arctic Ocean')
    elif ao3.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        aomask[i,j]=1.
        #print('Arctic Ocean')
    elif ao4.geometry.contains(Point(xnew[i,j], ynew[i,j])).values[0]:
        aomask[i,j]=1.
        #print('Arctic Ocean')
        
    else:
        nonemask[i,j]=1.

In [12]:
for i in GOA_others:
    goamask[i[0],i[1]]=1.
for i in AO_others:
    aomask[i[0],i[1]]=1.
for i in BS_others:
    bsmask[i[0],i[1]]=1.

In [13]:
ds = xr.Dataset({
                 'land':(['lats','lons'],themask),
                 'goa':(['lats','lons'],goamask),
                 'bs':(['lats','lons'],bsmask),
                 'ao':(['lats','lons'],aomask),
                 'none':(['lats','lons'],nonemask),
                },
                 attrs={'Trajectories and File Author':'Maria J. Molina'})

In [14]:
ds

<xarray.Dataset>
Dimensions:  (lats: 48, lons: 52)
Dimensions without coordinates: lats, lons
Data variables:
    land     (lats, lons) float64 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0
    goa      (lats, lons) float64 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0 0.0
    bs       (lats, lons) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ao       (lats, lons) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    none     (lats, lons) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    Trajectories and File Author:  Maria J. Molina

In [16]:
ds.to_netcdf(f'/glade/scratch/molina/basile/maskfile.nc')